### ------ JC -------
 (a/b test - model ready)

In [5]:
import pandas as pd

df_ab = pd.read_csv("Datos procesados/df_ab_test.csv")
df_model = pd.read_csv("Datos procesados/df_model_ready.csv")

print("AB TEST:", df_ab.shape)
print("MODEL READY:", df_model.shape)


AB TEST: (50500, 14)
MODEL READY: (70609, 14)


In [6]:
print("AB TEST columns:")
print(df_ab.columns)

print("\nMODEL READY columns:")
print(df_model.columns)


AB TEST columns:
Index(['client_id', 'age_group', 'gender', 'tenure_months', 'num_accounts',
       'balance', 'balance_log', 'logins_6m', 'calls_6m', 'experiment_group',
       'web_events', 'web_visits', 'web_steps', 'web_confirms'],
      dtype='object')

MODEL READY columns:
Index(['client_id', 'age_group', 'gender', 'tenure_months', 'num_accounts',
       'balance', 'balance_log', 'logins_6m', 'calls_6m', 'experiment_group',
       'web_events', 'web_visits', 'web_steps', 'web_confirms'],
      dtype='object')


In [7]:
[c for c in df_model.columns if "step" in c.lower()]



['web_steps']

In [8]:
df_model[['web_steps','web_confirms']].head()


,web_steps,web_confirms
0,5,3
1,5,1
2,4,0
3,1,0
4,1,0


In [ ]:
# Crear variable converted
# calcular la tasa de conversión por grupo
df_model['converted'] = (df_model['web_confirms'] > 0).astype(int)

df_model.groupby('experiment_group')['converted'].mean()


experiment_group
Control          0.655873
No_experiment    0.680243
Test             0.692932
Name: converted, dtype: float64

In [10]:
# Filtrar solo Test y Control
df_exp = df_model[df_model['experiment_group'].isin(['Control','Test'])].copy()

df_exp['experiment_group'].value_counts()


experiment_group
Test       26968
Control    23532
Name: count, dtype: int64

In [11]:
# Entender la fricción
df_exp.groupby('experiment_group')['web_steps'].describe()


,count,mean,std,min,25%,50%,75%,max
experiment_group,,,,,,,,
Control,23532.0,4.039393,1.492099,1.0,3.0,5.0,5.0,5.0
Test,26968.0,4.181697,1.372992,1.0,4.0,5.0,5.0,5.0


In [ ]:
# necesitamos separar: Usuarios que convierten / Usuarios que no convierten
df_exp.groupby(['experiment_group','converted'])['web_steps'].mean()


experiment_group  converted
Control           0            2.262657
                  1            4.971621
Test              0            2.459365
                  1            4.944935
Name: web_steps, dtype: float64

### Qué nos dicen los números

- Usuarios que NO convierten (converted = 0)
- Control: 2.26 pasos promedio
- Test: 2.46 pasos promedio
- 👉 Los que abandonan se van muy temprano.
- 👉 En ambos grupos, el abandono ocurre alrededor del paso 2.

- Usuarios que SÍ convierten (converted = 1)
- Control: 4.97 pasos
- Test: 4.94 pasos
- 👉 Los que convierten casi siempre llegan al final (≈ 5 pasos).
- 👉 No hay diferencia real aquí entre Test y Control.

### El Test:

- NO reduce pasos
- NO cambia cuántos pasos hacen los que convierten
- NO cambia mucho cuántos pasos hacen los que abandonan

###  Pero…

- 👉 AUMENTA la proporción de personas que pasan de “abandono” a “conversión”.
- Eso significa que el rediseño no cambia el comportamiento dentro del flujo.
- Cambia la decisión de seguir.
- Eso es mucho más interesante.

###  Insight

- El abandono ocurre principalmente en los primeros 2–3 pasos en ambos grupos.
- El rediseño no reduce la fricción estructural del flujo, pero aumenta la probabilidad de que los usuarios continúen hasta el final.

- Es decir:
- 👉 El problema no es la parte final del funnel.
- 👉 La fricción real está al inicio.
- 👉 El Test mejora el arranque, no el cierre.